Synthetic data, one warehouse lost demand

In [2]:
%load_ext autoreload
%autoreload 2

import analysis.ray_results_interpreter as rri
from importlib import reload
import pandas as pd

In [17]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/3",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/20",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/ctx/50"
}
results_interpretor = rri.RayResultsinterpreter()
conditions = {'context': [0, 1, 256]}
df_ctx = results_interpretor.make_table(paths, conditions)
df_ctx.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

min_test_loss = df_ctx.groupby(['# of stores'])['Test Loss'].transform('min')
df_ctx_print = df_ctx.copy()
df_ctx_print['Test Gap %'] = ((df_ctx_print['Test Loss'] - min_test_loss) / min_test_loss) * 100
print(df_ctx_print.to_string(index=False))

KeyboardInterrupt: 

In [3]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/3",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/20",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN/50"
}

results_interpretor = rri.RayResultsinterpreter()
conditions = {'for_all_networks': [1, 16, 64]}
df_gnn = results_interpretor.make_table(paths, conditions)
df_gnn.rename(columns={'for_all_networks': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

min_test_loss = df_gnn.groupby(['# of stores'])['Test Loss'].transform('min')
df_gnn_print = df_gnn.copy()
df_gnn_print['Test Gap %'] = ((df_gnn_print['Test Loss'] - min_test_loss) / min_test_loss) * 100
print("Architecture Class: GNN")
print(df_gnn_print.to_string(index=False))

Architecture Class: GNN
 # of stores  context size  Learning Rate  Train Loss  Dev Loss  Test Loss  Test Gap %
           3             1         0.0100    5.614032  5.609062   5.610129    0.011686
           3            16         0.0100    5.618455  5.607520   5.609473    0.000000
           3            64         0.0010    5.613282  5.608445   5.609535    0.001103
          10             1         0.0100    5.777970  5.816457   5.784590    1.175626
          10            16         0.0100    5.708765  5.751087   5.717375    0.000000
          10            64         0.0100    5.710389  5.751652   5.717664    0.005067
          20             1         0.0010    5.939852  5.911295   5.890847    1.350903
          20            16         0.0100    5.841372  5.834482   5.812328    0.000000
          20            64         0.0100    5.843079  5.833272   5.812664    0.005784
          50             1         0.0001    7.469758  7.450507   9.589425   78.380294
          50       

In [4]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/3",
    10: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/10",
    20: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/20",
    50: "/user/ml4723/Prj/NIC/ray_results/perf/GNN_message_passing/50"
}
results_interpretor = rri.RayResultsinterpreter()
conditions = {'for_all_networks': [1, 16, 64]}
df_gnn_mp = results_interpretor.make_table(paths, conditions)
df_gnn_mp.rename(columns={'for_all_networks': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

min_test_loss = df_gnn_mp.groupby(['# of stores'])['Test Loss'].transform('min')
df_gnn_mp_print = df_gnn_mp.copy()
df_gnn_mp_print['Test Gap %'] = ((df_gnn_mp_print['Test Loss'] - min_test_loss) / min_test_loss) * 100
print("Architecture Class: GNN Message Passing")
print(df_gnn_mp_print.to_string(index=False))

Architecture Class: GNN Message Passing
 # of stores  context size  Learning Rate  Train Loss  Dev Loss  Test Loss  Test Gap %
           3             1         0.0100    5.620989  5.612162   5.614612    0.107264
           3            16         0.0100    5.613380  5.608589   5.608596    0.000000
           3            64         0.0010    5.613815  5.608413   5.609913    0.023482
          10             1         0.0100    5.721958  5.761270   5.726167    0.151508
          10            16         0.0100    5.711121  5.752939   5.717505    0.000000
          10            64         0.0001    5.717989  5.757410   5.722908    0.094494
          20             1         0.0010    5.852099  5.842484   5.821487    0.000000
          20            16         0.0010    5.853936  5.845516   5.824511    0.051936
          20            64         0.0010    5.842828  5.844684   5.822395    0.015586
          50             1         0.0100    8.102721  8.089805  10.459843   95.248555
   

In [59]:
df_ctx.insert(1, 'Architecture Class', "Symmetry_Aware")
df_ctx = df_ctx.loc[df_ctx.groupby(['# of stores'])['Dev Loss'].idxmin()]

df_gnn.insert(1, 'Architecture Class', "GNN")
df_gnn = df_gnn.loc[df_gnn.groupby(['# of stores'])['Dev Loss'].idxmin()]

df_gnn_mp.insert(1, 'Architecture Class', "GNN Message Passing")
df_gnn_mp = df_gnn_mp.loc[df_gnn_mp.groupby(['# of stores'])['Dev Loss'].idxmin()]

In [60]:
vanilla = [
    {
                "# of stores": 3,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.610,
                "Dev Loss": 5.610,
                "Test Loss": 5.610,
            },
    {
                "# of stores": 10,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.720,
                "Dev Loss": 5.740,
                "Test Loss": 5.720,
            },
    {
                "# of stores": 20,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.850,
                "Dev Loss": 5.870,
                "Test Loss": 5.850,
            },
    {
                "# of stores": 50,
                "Architecture Class": "Vanilla",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": 5.410,
                "Dev Loss": 5.400,
                "Test Loss": 5.420,
            },
]
df_vanilla = pd.DataFrame(vanilla)

df = pd.concat([df_ctx, df_gnn, df_gnn_mp])
architecture_order = ['Symmetry_Aware', 'GNN', 'GNN Message Passing', 'Vanilla']
df['Architecture Class'] = pd.Categorical(df['Architecture Class'], categories=architecture_order, ordered=True)
min_test_loss = df.groupby(['# of stores'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100
df.sort_values(by=['# of stores', 'Architecture Class'], inplace=True)
print(df.to_string(index=False))

 # of stores  Architecture Class context size  Learning Rate  Train Loss  Dev Loss  Test Loss  Test Gap %
           3      Symmetry_Aware            1         0.0100    5.613812  5.608477   5.609850    0.006726
           3                 GNN           16         0.0100    5.618455  5.607520   5.609473    0.000000
           3 GNN Message Passing           64         0.0010    5.613815  5.608413   5.609913    0.007838
           3             Vanilla         None            NaN    5.610000  5.610000   5.610000    0.009392
          10      Symmetry_Aware            1         0.0100    5.712263  5.755829   5.720371    0.052401
          10                 GNN           16         0.0100    5.708765  5.751087   5.717375    0.000000
          10 GNN Message Passing           16         0.0100    5.711121  5.752939   5.717505    0.002278
          10             Vanilla         None            NaN    5.720000  5.740000   5.720000    0.045917
          20      Symmetry_Aware            1 

Synthetic data, Transshipment

In [84]:
paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/10",
}
df_sym = make_the_result_table(paths, [0, 1, 256])
df_sym.insert(1, 'Architecture Class', "Symmetry_Aware")

paths = {
    3: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/3",
    5: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/5",
    10: "/user/ml4723/Prj/NIC/ray_results/transshipment/vanilla/10",
}
df_van = make_the_result_table(paths, [None])
df_van.insert(1, 'Architecture Class', "Vanilla")

lower_bound = [
    {
                "# of stores": 3,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 6.19,
            },
    {
                "# of stores": 5,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 5.75,
            },
    {
                "# of stores": 10,
                "Architecture Class": "Lower bound",
                "context size": None,
                "Learning Rate": None,
                "Train Loss": None,
                "Dev Loss": None,
                "Test Loss": 6.05,
            },
]
df_lower_bound = pd.DataFrame(lower_bound)

df = pd.concat([df_sym, df_van, df_lower_bound])
df.sort_values(by=['# of stores', 'context size'], inplace=True)
print(df.to_string(index=False))

 # of stores Architecture Class context size  Learning Rate  Train Loss  Dev Loss  Test Loss
           3     Symmetry_Aware            0         0.0100   15.786119 15.745235  38.493707
           3     Symmetry_Aware            1         0.0010    6.451584  6.442219   6.436544
           3     Symmetry_Aware          256         0.0100    6.215819  6.195688   6.190809
           3            Vanilla         None         0.0001    6.202121  6.195605   6.190790
           3        Lower bound         None            NaN         NaN       NaN   6.190000
           5     Symmetry_Aware            0         0.0010   14.106193 14.152811  34.056048
           5     Symmetry_Aware            1         0.0100    6.061970  6.045415   6.036085
           5     Symmetry_Aware          256         0.0010    5.751615  5.759176   5.751703
           5            Vanilla         None         0.0001    5.755991  5.759228   5.751869
           5        Lower bound         None            NaN         Na

One warehouse lost demand synthetic - different primitive setup

In [3]:
paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/ctx'
}

results_interpretor = rri.RayResultsinterpreter()
conditions = {'context': [0, 1, 16, 64], 'warehouse_holding_cost': [0.7, 1.0, 1.3, 2.0], 'warehouse_lead_time': [6], 'stores_correlation': [0.5]}
df = results_interpretor.make_table(paths, conditions)
df.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)
min_test_loss = df.groupby(['warehouse_holding_cost', 'warehouse_lead_time', 'stores_correlation'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100
df.sort_values(by=['warehouse_holding_cost', 'warehouse_lead_time', 'stores_correlation', 'context size'], inplace=True)
print(df.to_string(index=False))

 # of stores  context size  warehouse_holding_cost  warehouse_lead_time  stores_correlation  Learning Rate  Train Loss  Dev Loss  Test Loss  Test Gap %
           3             0                     0.7                    6                 0.5          0.010    5.913846  5.907117   5.913744    1.509709
           3             1                     0.7                    6                 0.5          0.001    5.843374  5.837429   5.838852    0.224195
           3            16                     0.7                    6                 0.5          0.001    5.829509  5.826976   5.827758    0.033756
           3            64                     0.7                    6                 0.5          0.001    5.824574  5.827352   5.825791    0.000000
           3             0                     1.0                    6                 0.5          0.010    6.003209  5.989507   6.003050    2.219561
           3             1                     1.0                    6                 

In [4]:
ctx_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/ctx/3',
    10: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/ctx/10',
    20: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/ctx/20',
    50: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/ctx/50'
}
vanilla_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/vanilla/3',
    10: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/vanilla/10',
    20: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/vanilla/20',
    50: '/user/ml4723/Prj/NIC/ray_results/diff_primitive/vanilla/50'
}

def custom_data_filler(out_row, reference_row):
    out_row['best_train_loss'] = reference_row['best_train_loss']

# shared_conditions = {'warehouse_holding_cost': [0.7], 'warehouse_lead_time': [2], 'stores_correlation': [0.5]}
# shared_conditions = {'warehouse_holding_cost': [0.7], 'warehouse_lead_time': [6], 'stores_correlation': [0.5]}
# shared_conditions = {'warehouse_holding_cost': [1.0], 'warehouse_lead_time': [2], 'stores_correlation': [0.5]}
# shared_conditions = {'warehouse_holding_cost': [1.0], 'warehouse_lead_time': [6], 'stores_correlation': [0.5]}

condition_for_ctx = shared_conditions.copy()
condition_for_ctx['context'] = [0, 1, 64]
results_interpretor = rri.RayResultsinterpreter()
df_ctx = results_interpretor.make_table(ctx_paths, condition_for_ctx, custom_data_filler)
df_ctx.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

df_vanilla = results_interpretor.make_table(vanilla_paths, shared_conditions, custom_data_filler)
df_vanilla.rename(columns={'learning_rate': 'Learning Rate'}, inplace=True)

df_ctx.insert(1, 'Architecture Class', "Symmatry_Aware")
df_vanilla.insert(1, 'Architecture Class', "Vanilla")
df = pd.concat([df_ctx, df_vanilla])
min_test_loss = df.groupby(['# of stores', 'warehouse_holding_cost', 'warehouse_lead_time', 'stores_correlation'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100
df.insert(df.columns.get_loc(df.columns[-2]), 'Test Gap %', df.pop('Test Gap %'))
df.drop(columns=['warehouse_holding_cost', 'warehouse_lead_time', 'stores_correlation'], inplace=True)
df.sort_values(by=['# of stores', 'context size'], inplace=True)

print(shared_conditions)
print(df.to_string(index=False))

{'warehouse_holding_cost': [0.7], 'warehouse_lead_time': [6], 'stores_correlation': [0.5]}
 # of stores Architecture Class  context size  Learning Rate  Train Loss  Dev Loss  Test Loss  # of runs  Test Gap %  best_train_loss
           3     Symmatry_Aware           0.0         0.0100    5.913846  5.907117   5.913744         13    1.498100         5.910705
           3     Symmatry_Aware           1.0         0.0100    5.838516  5.833413   5.832253         12    0.099469         5.834330
           3     Symmatry_Aware          64.0         0.0010    5.830711  5.826821   5.826458         12    0.000000         5.825323
           3            Vanilla           NaN         0.0001    5.832769  5.832026   5.832205          6    0.098651         5.822270
          10     Symmatry_Aware           0.0         0.0010    6.008479  6.030406   6.007258          6    1.818944         5.996368
          10     Symmatry_Aware           1.0         0.0010    5.906434  5.947268   5.906385          6 

In [24]:
ctx_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/new_perf/ctx/3',
    5: '/user/ml4723/Prj/NIC/ray_results/new_perf/ctx/5',
    20: '/user/ml4723/Prj/NIC/ray_results/new_perf/ctx/20',
    50: '/user/ml4723/Prj/NIC/ray_results/new_perf/ctx/50',
    100: '/user/ml4723/Prj/NIC/ray_results/new_perf/ctx/100'
}
vanilla_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/new_perf/vanilla/3',
    5: '/user/ml4723/Prj/NIC/ray_results/new_perf/vanilla/5',
    20: '/user/ml4723/Prj/NIC/ray_results/new_perf/vanilla/20',
    50: '/user/ml4723/Prj/NIC/ray_results/new_perf/vanilla/50',
    100: '/user/ml4723/Prj/NIC/ray_results/new_perf/vanilla/100'
}

def custom_data_filler(out_row, reference_row):
    out_row['best_train_loss'] = reference_row['best_train_loss']

results_interpretor = rri.RayResultsinterpreter()
df_ctx = results_interpretor.make_table(ctx_paths, {'context': [0, 1, 2, 4, 8, 16, 32, 64]}, custom_data_filler)
df_ctx.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

df_vanilla = results_interpretor.make_table(vanilla_paths, {}, custom_data_filler)
df_vanilla.rename(columns={'learning_rate': 'Learning Rate'}, inplace=True)

df_ctx.insert(1, 'Architecture Class', "Symmatry_Aware")
df_vanilla.insert(1, 'Architecture Class', "Vanilla")
df = pd.concat([df_ctx, df_vanilla])
min_test_loss = df.groupby(['# of stores'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100
df.insert(df.columns.get_loc(df.columns[-2]), 'Test Gap %', df.pop('Test Gap %'))
df.sort_values(by=['# of stores', 'context size'], inplace=True)

print(df.to_string(index=False))

# for 3, 9 from 512 512 512, 0.01, 0.001, 0.0001
# 10 from 512, 512, 512, 0.001, 0.0001
# 20 from 128, 128, 128, 0.01, 0.001


 # of stores Architecture Class  context size  Learning Rate  Train Loss  Dev Loss  Test Loss  # of runs  Test Gap %  best_train_loss
           3     Symmatry_Aware           0.0         0.0100    5.678947  5.667238   5.658443          9    3.297811         5.665088
           3     Symmatry_Aware           1.0         0.0100    5.494950  5.499710   5.489791          9    0.218976         5.487189
           3     Symmatry_Aware          32.0         0.0010    5.475683  5.493100   5.478564          9    0.014024         5.472106
           3     Symmatry_Aware          64.0         0.0001    5.476005  5.493236   5.477796          9    0.000000         5.474725
           3            Vanilla           NaN         0.0010    5.485427  5.499762   5.493346         39    0.283868         5.472646
           5     Symmatry_Aware           0.0         0.0001    5.292834  5.278422   5.309792          7    3.518935         5.292628
           5     Symmatry_Aware           1.0         0.0010  

In [5]:
ctx_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/new_perf/ctx_large/3',
}
vanilla_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/new_perf/vanilla_large/3',
}

def custom_data_filler(out_row, reference_row):
    out_row['best_train_loss'] = reference_row['best_train_loss']
results_interpretor = rri.RayResultsinterpreter()
df_ctx = results_interpretor.make_table(ctx_paths, {'context': [0, 1, 64]}, custom_data_filler)
df_ctx.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

df_vanilla = results_interpretor.make_table(vanilla_paths, {}, custom_data_filler)
df_vanilla.rename(columns={'learning_rate': 'Learning Rate'}, inplace=True)

df_ctx.insert(1, 'Architecture Class', "Symmatry_Aware")
df_vanilla.insert(1, 'Architecture Class', "Vanilla")
df = pd.concat([df_ctx, df_vanilla])
min_test_loss = df.groupby(['# of stores'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100
df.insert(df.columns.get_loc(df.columns[-2]), 'Test Gap %', df.pop('Test Gap %'))
df.sort_values(by=['# of stores', 'context size'], inplace=True)

print(df.to_string(index=False))

# for 3, 9 from 512 512 512, 0.01, 0.001, 0.0001
# 10 from 512, 512, 512, 0.001, 0.0001
# 20 from 128, 128, 128, 0.01, 0.001


 # of stores Architecture Class  context size  Learning Rate  Train Loss  Dev Loss  Test Loss  # of runs  Test Gap %  best_train_loss
           3     Symmatry_Aware           0.0          0.001    5.677065  5.668593   5.659001          5    3.401764         5.670380
           3     Symmatry_Aware           1.0          0.010    5.484450  5.489308   5.473140          5    0.005710         5.484236
           3     Symmatry_Aware          64.0          0.001    5.480534  5.488657   5.472828          5    0.000000         5.477936
           3            Vanilla           NaN          0.001    5.481088  5.490378   5.477924         20    0.093108         5.473786


In [4]:
ctx_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/new_perf/ctx_lead/3',
}
vanilla_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/new_perf/vanilla_lead/3',
}

def custom_data_filler(out_row, reference_row):
    out_row['best_train_loss'] = reference_row['best_train_loss']
results_interpretor = rri.RayResultsinterpreter()
df_ctx = results_interpretor.make_table(ctx_paths, {'context': [0, 1, 64]}, custom_data_filler)
df_ctx.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

df_vanilla = results_interpretor.make_table(vanilla_paths, {}, custom_data_filler)
df_vanilla.rename(columns={'learning_rate': 'Learning Rate'}, inplace=True)

df_ctx.insert(1, 'Architecture Class', "Symmatry_Aware")
df_vanilla.insert(1, 'Architecture Class', "Vanilla")
df = pd.concat([df_ctx, df_vanilla])
min_test_loss = df.groupby(['# of stores'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100
df.insert(df.columns.get_loc(df.columns[-2]), 'Test Gap %', df.pop('Test Gap %'))
df.sort_values(by=['# of stores', 'context size'], inplace=True)

print(df.to_string(index=False))

# for 3, 9 from 512 512 512, 0.01, 0.001, 0.0001
# 10 from 512, 512, 512, 0.001, 0.0001
# 20 from 128, 128, 128, 0.01, 0.001


 # of stores Architecture Class  context size  Learning Rate  Train Loss  Dev Loss  Test Loss  # of runs  Test Gap %  best_train_loss
           3     Symmatry_Aware           0.0          0.010    5.924473  5.930560   5.910807          9    1.531933         5.907070
           3     Symmatry_Aware           1.0          0.010    5.834254  5.850992   5.821624          9    0.000000         5.823745
           3     Symmatry_Aware          64.0          0.010    5.836678  5.855047   5.828812          9    0.123483         5.809814
           3            Vanilla           NaN          0.001    5.836746  5.855643   5.823750         25    0.036534         5.796001


In [3]:
ctx_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/new_perf/ctx_large_lead/3',
}
vanilla_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/new_perf/vanilla_large_lead/3',
}

def custom_data_filler(out_row, reference_row):
    out_row['best_train_loss'] = reference_row['best_train_loss']
results_interpretor = rri.RayResultsinterpreter()
df_ctx = results_interpretor.make_table(ctx_paths, {'context': [0, 1, 64]}, custom_data_filler)
df_ctx.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

df_vanilla = results_interpretor.make_table(vanilla_paths, {}, custom_data_filler)
df_vanilla.rename(columns={'learning_rate': 'Learning Rate'}, inplace=True)

df_ctx.insert(1, 'Architecture Class', "Symmatry_Aware")
df_vanilla.insert(1, 'Architecture Class', "Vanilla")
df = pd.concat([df_ctx, df_vanilla])
min_test_loss = df.groupby(['# of stores'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100
df.insert(df.columns.get_loc(df.columns[-2]), 'Test Gap %', df.pop('Test Gap %'))
df.sort_values(by=['# of stores', 'context size'], inplace=True)

print(df.to_string(index=False))

# for 3, 9 from 512 512 512, 0.01, 0.001, 0.0001
# 10 from 512, 512, 512, 0.001, 0.0001
# 20 from 128, 128, 128, 0.01, 0.001


 # of stores Architecture Class  context size  Learning Rate  Train Loss  Dev Loss  Test Loss  # of runs  Test Gap %  best_train_loss
           3     Symmatry_Aware           0.0          0.010    5.939833  5.923935   5.904784          9    1.543459         5.921181
           3     Symmatry_Aware           1.0          0.010    5.845350  5.842640   5.815340          9    0.005311         5.841207
           3     Symmatry_Aware          64.0          0.010    5.837446  5.845638   5.815469          9    0.007529         5.836145
           3            Vanilla           NaN          0.001    5.847357  5.845670   5.815031         10    0.000000         5.836385


In [6]:
ctx_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/stable_bench/ctx/3',
}
vanilla_paths = {
    3: '/user/ml4723/Prj/NIC/ray_results/stable_bench/vanilla/3',
}

def custom_data_filler(out_row, reference_row):
    out_row['best_train_loss'] = reference_row['best_train_loss']
results_interpretor = rri.RayResultsinterpreter()
df_ctx = results_interpretor.make_table(ctx_paths, {'context': [0, 1, 64]}, custom_data_filler)
df_ctx.rename(columns={'context': 'context size', 'learning_rate': 'Learning Rate'}, inplace=True)

df_vanilla = results_interpretor.make_table(vanilla_paths, {}, custom_data_filler)
df_vanilla.rename(columns={'learning_rate': 'Learning Rate'}, inplace=True)

df_ctx.insert(1, 'Architecture Class', "Symmatry_Aware")
df_vanilla.insert(1, 'Architecture Class', "Vanilla")
df = pd.concat([df_ctx, df_vanilla])
min_test_loss = df.groupby(['# of stores'])['Test Loss'].transform('min')
df['Test Gap %'] = ((df['Test Loss'] - min_test_loss) / min_test_loss) * 100
df.insert(df.columns.get_loc(df.columns[-2]), 'Test Gap %', df.pop('Test Gap %'))
df.sort_values(by=['# of stores', 'context size'], inplace=True)

print(df.to_string(index=False))


 # of stores Architecture Class  context size  Learning Rate  Train Loss  Dev Loss  Test Loss  # of runs  Test Gap %  best_train_loss
           3     Symmatry_Aware           0.0          0.010    5.694042  5.685507   5.662421          3    3.401921         5.683701
           3     Symmatry_Aware           1.0          0.010    5.551475  5.565333   5.562188          3    1.571570         5.551475
           3     Symmatry_Aware          64.0          0.010    5.490902  5.505724   5.476127          2    0.000000         5.490902
           3            Vanilla           NaN          0.001    5.519690  5.518813   5.501438         12    0.462204         5.519690
